# DATASET PREPROCESSING

## READING CSV AND REMOVING USELESS RECORDS/COLUMNS

In [ ]:
import pandas as pd
df = pd.read_csv("en_lyrics.csv")
df = df.loc[df["tag"] != "misc"].drop(['year', 'language', 'language_cld3', 'language_ft', 'id'], axis=1)


In [2]:
df.head()

,title,tag,artist,views,features,lyrics
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki..."
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,..."
2,Forgive Me Father,rap,Fabolous,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...
3,Down and Out,rap,Cam'ron,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...
4,Fly In,rap,Lil Wayne,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ..."


In [3]:
df["tag"].unique()

array(['rap', 'rb', 'rock', 'pop', 'country'], dtype=object)

In [4]:
df.tail()

,title,tag,artist,views,features,lyrics
3374193,Everything Is Alright Now,pop,Chuck Bernard,2,{},"Everything is alright now\nOh yes, baby\nEvery..."
3374194,White Lies,pop,ElementD,1,"{""Harley Bird""}",[Verse 1]\nHalf truth and half you\nDidn't we ...
3374195,Ocean,pop,Effemar,3,{},[Verse 1]\nDance for me now\nKeeping yourself ...
3374196,Raise Our Hands,pop,"Culture Code, Pag & Mylo",3,"{Elex,""Culture Code / Pag & Mylo""}",[Verse 1]\nHere our purpose feels alive\nWe ar...
3374197,New Number,country,Alana Springsteen,1,{},"[Verse 1]\nYou need a new number, one that ain..."


## HOW TO CHECK THE STANZAS SPLITTING

In [5]:
#canzoni che pare non abbiano nessun modo per dividere in strofe
count = 0
for ly in df["lyrics"]:
    if "Verse" not in ly and "verse" not in ly and "\n\n" not in ly:
        count += 1
count


290203

In [6]:
#restituisce indice di canzoni che matchano la condizione
#ossia che non hanno "Verse", "verse" and \n\n
for idx, ly in enumerate(df["lyrics"]):
    if "Verse" not in ly and "verse" not in ly and "\n\n" not in ly:
        index_of_ly = idx  
        break
index_of_ly

160

In [7]:
df.loc[262, "lyrics"]

"Black people raise up, anger management cases to the front line\nYour day's up, children, I hope you didn't quit your day jobs\nBeen in it, hosting shit like Canadian Asians host SARS\nI'm kidding, no really pa, you're fibbing, you rented your car\nI'm spilling these phrases like seizure waitresses holding trays\nThis is greatness, the most endangered species on the playlist\nHey pay this, you fucks\nNo need to pay for a facelift you sluts\nYou slept, stupid I'm more dangerous than Michael's face is\nI cut through it like a machete's built on my tongue\nLeave you stitched up like Eeyore with your teeth all gums, you weak whore\nThe meek inherits now, but I speak veteran prose\nLike I'm Tom Cruise in a wheelchair with my pants all sewn\nNow how many times do I need to explain\nYou faggot Hannibal Lector niggas, you all eat brain\nI will damage your whole system, hack into your mainframe\nCrack your bones like I'm cracking computer codes, I am not playing\nI am not done spraying nigga, 

## SPLITTING STROFE

In [33]:
for ly in df["lyrics"]:
    print(type(ly))
    break

<class 'str'>


In [ ]:
# Function to process each lyric
import re

#da sistemare mettendo nella regex i nomi dei separatori più importanti; come controllare i typo?
#oppure rimanere più generici settando una regex tipo r"\[.*\]""
def process_lyric(ly):
    #pattern = r"\[(Introduction|Interlude|Coda|Hook|Prehook|Posthook|Conclusion|Refrain|Verse|Intro|Outro|Chorus|Bridge)\s*\w*\]" #qui mi ha balzato dei [Verse]
    pattern = r"\[(Introduction|Interlude|Coda|Hook|Prehook|Posthook|Conclusion|Refrain|Verse|Intro|Outro|Chorus|Bridge).*\]"
    if re.search(pattern, ly):
        return ly.strip().split("\n[") #qui è giusto tenere \n prima della quadra?
    elif "\n\n" in ly: #and interlude, verse, coda ecc. not in ly:
        return ly.strip().split("\n\n")
    return [ly]  # Default case: no splitting

# Apply the function to the lyrics column
df["processed_lyrics"] = df["lyrics"].apply(process_lyric)


In [4]:
#check the "processed_lyrics" attribute
df.head()

,title,tag,artist,views,features,lyrics,processed_lyrics
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...","[[Chorus: Opera Steve & Cam'ron]\nKilla Cam, K..."
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...","[[Produced by Irv Gotti]\n, Intro]\nYeah, hah,..."
2,Forgive Me Father,rap,Fabolous,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,[Maybe cause I'm eatin\nAnd these bastards fie...
3,Down and Out,rap,Cam'ron,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,"[[Produced by Kanye West and Brian Miller], [I..."
4,Fly In,rap,Lil Wayne,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...","[[Intro]\nSo they ask me\n""Young boy\nWhat you..."


In [ ]:
#output è una lista di stringhe
x = df.loc[123, "processed_lyrics"] 
x

["[Intro: Lil Jon]\nWhat, uh\nWhat?! You can't! You can't\nIt's Trillville ho, Lil Scrappy ho, it's BME ho\nLet's go!\n",
 'Refrain (repeated throughout song)]\nGet on my level ho\n',
 'Chorus: Lil Jon]\nBitch nigga you can neva eva\nEva eva\nEva eva, eva eva eva eva\nGet on my level ho (What!)\nGet on my level ho (What!)\nGet on my level ho (What!)\nGet on my level ho (What!)\nBitch nigga you can neva eva\nEva eva\nEva eva, eva eva eva eva\nGet on my level ho (What!)\nGet on my level ho (What!)\nGet on my level ho (What!)\nGet on my level ho (What!)',
 "Verse 1: Dirty Mouth]\nGet the fuck back, get the fuck back already\nI ain't playing nann day\nProve a nigga what you say\nMeet me anywhere, I don't care\nIt don't matter to me\nCause if you fucking step\nI'mma step in your vicinity\nWhat, you think I'm a ho?\nNigga please!\nYou ain't on my level\nGet some rank, bitch\nWhile I'm digging your grave with a shovel\nTime after time I ask myself\nIs it really clever for you to get with me?\

### cosa importante notata: già genius di per sè se c'è un hook uguale riporta solo [hook] nel lyrics e non lo riscrive tutto;
### ecco perchè in x = df.loc[2, "processed_lyrics"] c'è una stringa (con indice 3) che è solo 'Hook]\n'
### questo si applica anche per i chorus?

In [12]:
len(x)

11

### a questo punto gestiamo prima la questione chorus uguali e poi facciamo ulteriore split così che rimangano le stringe pulite

## Metodo per fare n record per le n strofe di una canzone

In [21]:
df_cut = df.iloc[0:3].reset_index(drop=True)
df_cut

,title,tag,artist,views,features,lyrics,processed_lyrics
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...","[[Chorus: Opera Steve & Cam'ron]\nKilla Cam, K..."
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...","[[Produced by Irv Gotti]\n, Intro]\nYeah, hah,..."
2,Forgive Me Father,rap,Fabolous,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,[Maybe cause I'm eatin\nAnd these bastards fie...


In [22]:
#controllare tipo di dato in attributo; explode funziona solo su stringhe
for ly in df["processed_lyrics"]:
    print(type(ly))
    break

<class 'list'>


In [23]:
exploded_df = df_cut.explode('processed_lyrics', ignore_index=False)
exploded_df.rename(columns={'processed_lyrics': 'stanza'}, inplace=True)

exploded_df

#come abbiamo fatto per processed lyrics facciamo per aggiungere colonna isChorus controllando se strofa inizia con stringa "Chorus"
#se inizia con chorus allora valore T else F (0,1)
#questione chorus uguali? da gestire prima di fare nuovo exploded df
#pulizia \n quadre con regex così che ogni stringa inizi pulita (e droppi quadre vuote dentro)
#tokenizza, lemmatizzazione, lowercasing, 


,title,tag,artist,views,features,lyrics,stanza
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki..."
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...","Verse 1]\nWith the goons I spar, stay in tune ..."
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",Chorus]\nKilla Cam\nKilla Cam Cam (sing)\nKill...
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...","Verse 2]\nYo, I'm from where Nicky Barnes got ..."
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",Chorus]\nKilla Cam (sing)\nKilla Cam Cam (clap...
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",Verse 3]\n(Killa!) Yo\nHow dope is this?\nTeac...
0,Killa Cam,rap,Cam'ron,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",Chorus]\nKilla Cam\nKilla Cam Cam\nKilla Cam K...
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",[Produced by Irv Gotti]\n
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...","Intro]\nYeah, hah, yeah, Roc-A-Fella\nWe invit..."
1,Can I Live,rap,JAY-Z,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",Verse 1]\nWhile I'm watchin' every nigga watch...


#### COSE DA FARE PRIMA DEL PREPROCESSING SUL TESTO VERO E PROPRIO
- regex per trovare altre tipologie di divisione strofe (per ora: intro, outro, chorus, verse, hook, bridge) e in generale per vedere cosa c'è tra quadre
- splittare anche secondo diciture sopra + \n\n[ ]
- regex per cancellare cose dentro le quadre (sia produced by... che i numeri che rimangono dallo split)

In [ ]:
#PUNTO 1

import re

pattern = r"\[\w+\]"
all_matches = []

for ly in df["processed_lyrics"]:  
    for i in ly: #per ogni stringa 
        matches = re.findall(pattern, i)
        for match in matches:
            if match not in all_matches:
                all_matches.extend(matches)

all_matches
#output è lista di ogni cosa che matcha questo pattern
#utile perchè così vediamo quali altri indicatori di strofe ci sono e quindi possiamo tenerli in considerazione per splitting strofe

['[Chorus]',
 '[Intro]',
 '[Hook]',
 '[Intro]',
 '[Verse]',
 '[Hook]',
 '[Outro]',
 '[HOOK]',
 '[Blaze]',
 '[Sniffing]',
 '[Verse]',
 '[Skit]',
 '[Talking]',
 '[Hook]',
 '[Hook]',
 '[Singing]',
 '[Talking]',
 '[Intro]',
 '[Refrain]',
 '[Verse]',
 '[Refrain]',
 '[Bridge]',
 '[unintelligible]',
 '[Break]',
 '[Chorus]',
 '[Outro]',
 '[Fades]',
 '[Chorus]',
 '[Chorus]',
 '[Coda]',
 '[Chorus]',
 '[What]',
 '[Applause]',
 '[Interlude]',
 '[Applause]',
 '[Outro]',
 '[Applause]',
 '[Hook]',
 '[Puffy]',
 '[Chorus]',
 '[Instrumental]',
 '[Outro]',
 '[Belo]',
 '[Twista]',
 '[I]',
 '[RZA]',
 '[Streetlife]',
 '[RFG]',
 '[DR]',
 '[Hook]',
 '[RFG]',
 '[RFG]',
 '[JPG]',
 '[DR]',
 '[JPG]',
 '[RFG]',
 '[JPG]',
 '[RFG]',
 '[JPG]',
 '[Intro]',
 '[Incomprehensible]',
 '[AMG]',
 '[Fabolous]',
 '[Hook]',
 '[Hook]',
 '[Boots]',
 '[Hook]',
 '[Juelz]',
 '[Banks]',
 '[Blackmel]',
 '[Esau]',
 '[LaShonda]',
 '[echoes]',
 '[echoes]',
 '[Scratching]',
 '[Sample]',
 '[Dres]',
 '[2Pac]',
 '[Repeat]',
 '[Eminem]',
 '[E

## PREPROCESSING 
### tutto quello che segue è sbagliato, lo teniamo comunque come appoggio ma da rivedere tutto

In [ ]:
#librerie inizio gestione preprocessing
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
#se le stopwords "ufficiali" sono solo queste come facciamo?
stopwords = set(stopwords.words("english"))


In [ ]:
ly1 = nltk.word_tokenize(df.loc[0, "lyrics"].lower())
for word in ly1:
    if word not in stopwords and word not in string.punctuation:
        print(word)
#come gestire parole che non rientrano nella lista di stopwords ma vanno comunque tolte
#es. ripetizioni

chorus
opera
steve
cam'ron
killa
cam
killa
cam
cam
killa
cam
killa
cam
killa
cam
cam
killa
cam
killa
cam
cam
killa
killa
killa
cam
killa
cam
cam
killa
killa
killa
cam
killa
cam
cam
bases
loaded
killa
cam
killa
cam
uh-huh
killa
cam
cam
santana
second
jim
third
killa
cam
killa
cam
cam
'm
bat
killa
killa
killa
cam
killa
cam
cam
killa
'm
'bout
hit
shit
world
killa
cam
ugh
heatmakerz
killa
cam
cam
killa
cam
killa
cam
killa
cam
cam
hahahaha
killa
cam
killa
cam
cam
killa
killa
killa
cam
killa
cam
cam
killa
make
shit
clap
killa
cam
killa
cam
cam
killa
cam
killa
cam
killa
cam
cam
killa
cam
killa
cam
cam
killa
killa
killa
cam
killa
killa
killa
cam
cam
killa
verse
1
goons
spar
stay
tune
like
``
damn
realest
since
'kumbaya
''
bomaye
killa
cam
lord
lord
still
man
pan
scrilla
fam
board
bitches
want
neuter
niggas
want
tutor
hooligan
houlihan
's
maneuvering
's
nothing
new
doggy
'm
land
grind
pan-pan
gram
dime
toes
mc
say
``
hammer
time
''
beef
hammer
mine
get
hands
nines
'bama
line
corduroys
cam
'll
s

In [ ]:
#modo un po' macchinoso per identificare la prima strofa
ly1 = nltk.word_tokenize(df.loc[0, "lyrics"].lower())
strofa1 = ""
for word in ly1:
    if word not in stopwords and word not in string.punctuation:
        strofa1 += word + " "
    if word == "verse":
        break
strofa1

"chorusoperastevecam'ronkillacamkillacamcamkillacamkillacamkillacamcamkillacamkillacamcamkillakillakillacamkillacamcamkillakillakillacamkillacamcambasesloadedkillacamkillacamuh-huhkillacamcamsantanasecondjimthirdkillacamkillacamcam'mbatkillakillakillacamkillacamcamkilla'm'bouthitshitworldkillacamughheatmakerzkillacamcamkillacamkillacamkillacamcamhahahahakillacamkillacamcamkillakillakillacamkillacamcamkillamakeshitclapkillacamkillacamcamkillacamkillacamkillacamcamkillacamkillacamcamkillakillakillacamkillakillakillacamcamkillaverse"